##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Apache ORC リーダー

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/orc"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>   </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

## 概要

Apache ORC は、一般的な列型ストレージ形式です。tensorflow-io パッケージは、[Apache ORC](https://orc.apache.org/) ファイルを読み取るデフォルトの実装を提供します。

## MNIST モデルをビルドする

必要なパッケージをインストールし、ランタイムを再起動する


In [2]:
!pip install tensorflow-io

In [3]:
import tensorflow as tf
import tensorflow_io as tfio

### ORC でサンプルデータセットファイルをダウンロードする

ここで使用するデータセットは、UCI の [Iris データセット](https://archive.ics.uci.edu/ml/datasets/iris)です。データセットには、それぞれ 50 インスタンスの 3 つのクラスが含まれ、各クラスはアヤメの植物のタイプを参照します。これには、（1）がく片の長さ、（2）がく片の幅、（3）花びらの長さ、（4）花びらの幅の、4つの属性があり、最後の列にはクラスラベルが含まれています。

In [4]:
!curl -OL https://github.com/tensorflow/io/raw/master/tests/test_orc/iris.orc
!ls -l iris.orc

## ファイルからデータセットを作成する

In [35]:
dataset = tfio.IODataset.from_orc("iris.orc", capacity=15).batch(1)

データセットを調べます。

In [42]:
for item in dataset.take(1):
    print(item)


Iris データセットに基づく ORC データセットを使用した tf.keras モデルトレーニングのエンドツーエンドの例を見ていきましょう。

### データの前処理をする

どの列が特徴で、どの列がラベルであるかを構成します。

In [47]:
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
label_cols = ["species"]

# select feature columns
feature_dataset = tfio.IODataset.from_orc("iris.orc", columns=feature_cols)
# select label columns
label_dataset = tfio.IODataset.from_orc("iris.orc", columns=label_cols)

モデルトレーニングのために種を浮動小数点数にマッピングする util 関数:

In [48]:
vocab_init = tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant(["virginica", "versicolor", "setosa"]),
    values=tf.constant([0, 1, 2], dtype=tf.int64))
vocab_table = tf.lookup.StaticVocabularyTable(
    vocab_init,
    num_oov_buckets=4)

In [49]:
label_dataset = label_dataset.map(vocab_table.lookup)
dataset = tf.data.Dataset.zip((feature_dataset, label_dataset))
dataset = dataset.batch(1)

def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features), axis=1)
    return features, labels

dataset = dataset.map(pack_features_vector)

## モデルをビルド、コンパイル、およびトレーニングする

これで、モデルを作成してトレーニングする準備が整いました。処理したデータセットからアヤメ植物のクラスを予測するために、3 つのレイヤーの keras モデルを構築します。

In [50]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            10, activation=tf.nn.relu, input_shape=(4,)
        ),
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(3),
    ]
)

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(dataset, epochs=5)